In [1]:
import os
import wave
import moviepy as mp
from vosk import Model, KaldiRecognizer

In [10]:
def extrair_audio(video_path, audio_path):
    """
    Extrai o áudio de um arquivo de vídeo usando moviepy e salva como WAV
    com 16 kHz, mono (codec pcm_s16le).
    """
    clip = mp.VideoFileClip(video_path)
    clip.audio.write_audiofile(
        audio_path, 
        codec='pcm_s16le', 
        fps=16000, 
        ffmpeg_params=["-ac", "1"]  # -ac 1 = força mono
    )

def reconhecer_fala_offline(audio_path, model_dir):
    """
    Faz a transcrição offline do arquivo de áudio usando o modelo Vosk.
    Retorna o texto transcrito.
    """
    # Carrega o modelo Vosk (isso pode demorar alguns segundos na primeira vez)
    model = Model(model_dir)

    # Abre o arquivo WAV para leitura
    with wave.open(audio_path, "rb") as wf:
        # Verifica se a taxa de amostragem está correta
        if wf.getframerate() != 16000:
            raise ValueError("A taxa de amostragem do áudio deve ser 16 kHz para este exemplo.")

        # Cria um reconhecedor Kaldi
        rec = KaldiRecognizer(model, wf.getframerate())
        
        texto_resultante = ""

        # Lê o áudio em blocos e alimenta o reconhecedor
        while True:
            data = wf.readframes(4000)  # 4000 frames ~ 0.25 seg a 16kHz
            if len(data) == 0:
                break

            if rec.AcceptWaveform(data):
                # Quando o reconhecedor aceita um "chunk" de áudio
                # podemos pegar o resultado parcial
                result_json = rec.Result()
                # Exemplo de como extrair apenas o "text" de result_json
                # mas geralmente a cada chunk não queremos concatenar tudo ainda
                # (porque é parcial)
                # Aqui vamos só deixar o Vosk montar o resultado final no rec.FinalResult()
                pass

        # Finaliza e pega resultado total
        final_json = rec.FinalResult()
        import json
        # final_json é algo como {"text": "texto final"}
        texto_resultante = json.loads(final_json).get("text", "")

    return texto_resultante

def processar_videos_em_pasta(pasta, model_dir):
    """
    Varre todos os arquivos .mp4 na pasta e realiza:
      - Extração do áudio em WAV (16 kHz, mono)
      - Transcrição offline via Vosk
      - Salva o texto em .txt com o mesmo nome do vídeo
    """
    for arquivo in os.listdir(pasta):
        if arquivo.lower().endswith(".mp4"):
            print(f"\nProcessando: {arquivo}")

            caminho_novo_audio = "C:\\Users\\josaf\\Documents\\Aulas\\aulas\\Ciencia-dados\\21-video-para-texto\\audio"
            caminho_novo_texto = "C:\\Users\\josaf\\Documents\\Aulas\\aulas\\Ciencia-dados\\21-video-para-texto\\texto"

            caminho_video = os.path.join(pasta, arquivo)
            nome_base = os.path.splitext(arquivo)[0]
            caminho_audio = os.path.join(caminho_novo_audio, f"{nome_base}.wav")
            caminho_txt = os.path.join(caminho_novo_texto, f"{nome_base}.txt")

            try:
                # 1) Extrair áudio
                extrair_audio(caminho_video, caminho_audio)
                print(f"Áudio extraído: {caminho_audio}")

                # 2) Reconhecer fala offline
                texto_transcrito = reconhecer_fala_offline(caminho_audio, model_dir)
                print(f"Transcrição: {texto_transcrito}")

                # 3) Salvar .txt
                with open(caminho_txt, "w", encoding="utf-8") as f:
                    f.write(texto_transcrito)
                print(f"Transcrição salva em: {caminho_txt}")

            except Exception as e:
                print(f"Erro ao processar {arquivo}: {e}")

def main():
    # Defina a pasta dos vídeos:
    pasta_videos = "D:\\Cursos\\Formacao Cientista de Dados\\07. Machine Learning\\01"
    # Defina onde está o modelo Vosk para português (ex.: "modelos/vosk-model-small-pt-0.4")
    model_dir = r"modelo-vosk/vosk-model-small-pt-0.3"

    if not os.path.isdir(pasta_videos):
        print(f"Pasta '{pasta_videos}' não existe.")
        return
    
    if not os.path.isdir(model_dir):
        print(f"Pasta de modelo '{model_dir}' não existe. Baixe e extraia o modelo Vosk para PT.")
        return

    processar_videos_em_pasta(pasta_videos, model_dir)

if __name__ == "__main__":
    main()



Processando: 01.01 Machine Learning.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf57.56.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [960, 540], 'bitrate': 311, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 117, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 74.77, 'bitrate': 437, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [960, 540], 'video_bitrate': 311, 'video_fps': 30.0

MoviePy - Done.
Áudio extraído: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\audio\01.01 Machine Learning.wav
Transcrição: 
Transcrição salva em: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\texto\01.01 Machine Learning.txt

Processando: 01.02 Introdução.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf57.56.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [960, 540], 'bitrate': 150, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 117, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0

MoviePy - Done.
Áudio extraído: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\audio\01.02 Introdução.wav
Transcrição: muito obrigado e até o próximo vídeo
Transcrição salva em: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\texto\01.02 Introdução.txt

Processando: 01.02.01 Formação Cientista de Dados.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf57.56.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [960, 524], 'bitrate': 150, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 117, 'metadata': {'Metadata': '', 'handle

MoviePy - Done.
Áudio extraído: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\audio\01.02.01 Formação Cientista de Dados.wav
Transcrição: 
Transcrição salva em: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\texto\01.02.01 Formação Cientista de Dados.txt

Processando: 01.03.01 O que é aprendizado de Máquina.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf57.56.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [960, 540], 'bitrate': 149, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 117, 'metadata': {'Metadata': '', 

MoviePy - Done.
Áudio extraído: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\audio\01.03.01 O que é aprendizado de Máquina.wav
Transcrição: à medida que se vai acontecendo tecnologia vai evoluindo bem agora somos capazes de criar sistemas mais inteligente falaremos mais sobre isso daqui a pouco
Transcrição salva em: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\texto\01.03.01 O que é aprendizado de Máquina.txt

Processando: 01.03.02 O que é aprendizado de Máquina.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf57.56.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [960, 540], 'bitrate': 149, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}

MoviePy - Done.
Áudio extraído: C:\Users\josaf\Documents\Aulas\aulas\Ciencia-dados\21-video-para-texto\audio\01.03.02 O que é aprendizado de Máquina.wav


KeyboardInterrupt: 

In [ ]:
from vosk import Model

model_path = r"modelo-vosk/vosk-model-small-pt-0.3" # Ajuste o caminho
model = Model(model_path)
print("Modelo carregado com sucesso!")

Modelo carregado com sucesso!


In [7]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')
